In [1]:
import requests
import time
from datetime import datetime
import pandas as pd
import os
import json
import uuid
import sqlite3
import sys
from dotenv import load_dotenv
# Load the .env file
load_dotenv()
# Access the environment variables
project_folder_path = os.getenv('PROJECT_FOLDER_PATH')
print(project_folder_path)
sys.path.append(project_folder_path)
from utils.db_functs import *
from utils.requests_functs import *
from utils.html_functs import *
from utils.pdf_functs import *
from utils.utils_functs import *

/home/roruizf/anaconda3/envs/data_env/lib/python3.9/site-packages/pandas/core/computation/expressions.py:21: UserWarning: Pandas requires version '2.8.0' or newer of 'numexpr' (version '2.7.3' currently installed).
  from pandas.core.computation.check import NUMEXPR_INSTALLED
/home/roruizf/anaconda3/envs/data_env/lib/python3.9/site-packages/pandas/core/arrays/masked.py:62: UserWarning: Pandas requires version '1.3.4' or newer of 'bottleneck' (version '1.3.2' currently installed).
  from pandas.core import (


/mnt/c/Users/rober/OneDrive/8_DEVELOPMENT/cev-data-lake


### Database and directories

In [2]:
db_file_name = 'cev-database-reports-v1.db'
db_file_path = os.path.join(project_folder_path, 'data', 'sqlite', db_file_name)

## 1) Define a dataframe with download coordinates
The dataframe must contain 1 row per `comuna` meaning 348 rows in total.

In [3]:
query = f"""
        SELECT * FROM reports_summary;
        """
print(query)


        SELECT * FROM reports_summary;
        


In [4]:
summary_df = create_dataframe_from_query(db_file_path, query)
summary_df.head()

,eval_id,comuna_id,region_id,tipo_evaluacion,version_evaluacion,pdf_file_name
0,01e92e4e-f6b7-514e-a256-06264f380675,113,1,1,2,1_113_1_01e92e4e-f6b7-514e-a256-06264f380675.pdf
1,0884c79b-e7a6-5c70-89b6-a54faafc65ec,113,1,1,1,1_113_1_0884c79b-e7a6-5c70-89b6-a54faafc65ec.pdf
2,0b05bbd6-37b3-5af0-a2a9-2efca779001f,113,1,1,1,1_113_1_0b05bbd6-37b3-5af0-a2a9-2efca779001f.pdf
3,14fbb359-7c44-5e81-9d3f-0d6fa2364b76,113,1,1,1,1_113_1_14fbb359-7c44-5e81-9d3f-0d6fa2364b76.pdf
4,16035bae-c656-5e37-b3f9-11e3c97a980c,113,1,1,1,1_113_1_16035bae-c656-5e37-b3f9-11e3c97a980c.pdf


Set additional features to track the status of downloads as well as times and file names

## 3) Check first if any pdf files has been downloaded
This is useful in case a seach has been run and has not completed

### 3.1) Read all html files for the corresponding `search_date` and `search_id`

In [6]:
# Define the directory path
pdf_files_dir = os.path.join(project_folder_path, 'data', 'raw', '3_evals_reports', 'pdf_files')

# Check if the directory exists
if not os.path.exists(pdf_files_dir):
    # If it doesn't exist, create the directory
    os.makedirs(pdf_files_dir)
    print(f"Directory '{pdf_files_dir}' created.")

# List all files in the directory
pdf_file_paths = find_pdf_files(pdf_files_dir)
print(f'The directory: {pdf_files_dir} contains {len(pdf_file_paths)} out of {summary_df.shape[0]}')

The directory: /mnt/c/Users/rober/OneDrive/8_DEVELOPMENT/cev-data-lake/data/raw/3_evals_reports/pdf_files contains 12383 out of 1314


### 3.2) Fill on dataframe all those rows corresponding to files already downloaded

### Scrape pdf files: 1 file per evaluacion

In [7]:
### PDF Summary

In [8]:
start_time = time.time()
informe_v2_pagina1_df = pd.DataFrame()
informe_v2_pagina2_df = pd.DataFrame()
informe_v2_pagina3_consumos_df = pd.DataFrame()
informe_v2_pagina3_envolvente_df = pd.DataFrame()
informe_v2_pagina4_df = pd.DataFrame()
informe_v2_pagina5_df = pd.DataFrame()
informe_v2_pagina6_df = pd.DataFrame()
informe_v2_pagina7_df = pd.DataFrame()

for index, row in summary_df.iterrows():
    print(f'Report {index+1} out of {summary_df.shape[0]}')
    
    # Check if report is v2
    pdf_file_path = os.path.join(project_folder_path, 'data', 'raw', '3_evals_reports', 'pdf_files', str(row['region_id']), str(row['pdf_file_name']))
    pdf_report = fitz.open(pdf_file_path)
    if str(row['version_evaluacion']) == '2':
        v2 = True
    else:
        v2 = False

    if v2:
        print(pdf_file_path)
        informe_v2_pagina1_df_i = scrape_informe_cev_v2_pagina1(pdf_file_path)
        informe_v2_pagina1_df_i.insert(0, 'eval_id', row['eval_id'])
        informe_v2_pagina1_df = pd.concat([informe_v2_pagina1_df, informe_v2_pagina1_df_i], axis=0)
        
        informe_v2_pagina2_df_i = scrape_informe_cev_v2_pagina2(pdf_file_path)
        informe_v2_pagina2_df_i.insert(0, 'eval_id', row['eval_id'])
        informe_v2_pagina2_df = pd.concat([informe_v2_pagina2_df, informe_v2_pagina2_df_i], axis=0)
        
        informe_v2_pagina3_consumos_df_i = scrape_informe_cev_v2_pagina3_consumos(pdf_file_path)
        informe_v2_pagina3_consumos_df_i.insert(0, 'eval_id', row['eval_id'])
        informe_v2_pagina3_consumos_df = pd.concat([informe_v2_pagina3_consumos_df, informe_v2_pagina3_consumos_df_i], axis=0)
        
        informe_v2_pagina3_envolvente_df_i = scrape_informe_cev_v2_pagina3_envolvente(pdf_file_path)
        informe_v2_pagina3_envolvente_df_i.insert(0, 'eval_id', row['eval_id'])
        informe_v2_pagina3_envolvente_df = pd.concat([informe_v2_pagina3_envolvente_df, informe_v2_pagina3_envolvente_df_i], axis=0)
        
        informe_v2_pagina4_df_i = scrape_informe_cev_v2_pagina4(pdf_file_path)
        informe_v2_pagina4_df_i.insert(0, 'eval_id', row['eval_id'])
        informe_v2_pagina4_df = pd.concat([informe_v2_pagina4_df, informe_v2_pagina4_df_i], axis=0)
        
        informe_v2_pagina5_df_i = scrape_informe_cev_v2_pagina5(pdf_file_path)
        informe_v2_pagina5_df_i.insert(0, 'eval_id', row['eval_id'])
        informe_v2_pagina5_df = pd.concat([informe_v2_pagina5_df, informe_v2_pagina5_df_i], axis=0)
        
        informe_v2_pagina6_df_i = scrape_informe_cev_v2_pagina6(pdf_file_path)
        informe_v2_pagina6_df_i.insert(0, 'eval_id', row['eval_id'])
        informe_v2_pagina6_df = pd.concat([informe_v2_pagina6_df, informe_v2_pagina6_df_i], axis=0)
        
        informe_v2_pagina7_df_i = scrape_informe_cev_v2_pagina7(pdf_file_path)        
        informe_v2_pagina7_df_i.insert(0, 'eval_id', row['eval_id'])
        informe_v2_pagina7_df = pd.concat([informe_v2_pagina7_df, informe_v2_pagina7_df_i], axis=0)
        
        # Insert into data base
        insert_unique_rows_from_dataframe(db_file_path, 'informe_v2_pagina1', informe_v2_pagina1_df_i, unique_columns=['eval_id'])
        insert_unique_rows_from_dataframe(db_file_path, 'informe_v2_pagina2', informe_v2_pagina2_df_i, unique_columns=['eval_id'])
        insert_unique_rows_from_dataframe(db_file_path, 'informe_v2_pagina3_consumos', informe_v2_pagina3_consumos_df_i, unique_columns=['eval_id'])
        insert_unique_rows_from_dataframe(db_file_path, 'informe_v2_pagina3_envolvente', informe_v2_pagina3_envolvente_df_i, unique_columns=['eval_id'])
        insert_unique_rows_from_dataframe(db_file_path, 'informe_v2_pagina4', informe_v2_pagina4_df_i, unique_columns=['eval_id'])
        insert_unique_rows_from_dataframe(db_file_path, 'informe_v2_pagina5', informe_v2_pagina5_df_i, unique_columns=['eval_id'])
        insert_unique_rows_from_dataframe(db_file_path, 'informe_v2_pagina6', informe_v2_pagina6_df_i, unique_columns=['eval_id'])
        insert_unique_rows_from_dataframe(db_file_path, 'informe_v2_pagina7', informe_v2_pagina7_df_i, unique_columns=['eval_id'])
        
        
informe_v2_pagina1_df.reset_index(drop=True, inplace=True)
informe_v2_pagina2_df.reset_index(drop=True, inplace=True)
informe_v2_pagina3_consumos_df.reset_index(drop=True, inplace=True)
informe_v2_pagina3_envolvente_df.reset_index(drop=True, inplace=True)
informe_v2_pagina4_df.reset_index(drop=True, inplace=True)
informe_v2_pagina5_df.reset_index(drop=True, inplace=True)
informe_v2_pagina6_df.reset_index(drop=True, inplace=True)
informe_v2_pagina7_df.reset_index(drop=True, inplace=True)
end_time = time.time()
execution_time = end_time - start_time
print("Script execution time:", execution_time, "seconds")

Report 1 out of 1314
/mnt/c/Users/rober/OneDrive/8_DEVELOPMENT/cev-data-lake/data/raw/3_evals_reports/pdf_files/10/10_143_2_bd213957-3ec3-5d4f-9b85-0278ceebccaf.pdf
Duplicate row found for unique columns ['eval_id']: ('bd213957-3ec3-5d4f-9b85-0278ceebccaf',). Skipping insertion.
Rows inserted into table 'informe_v2_pagina1' successfully.
Duplicate row found for unique columns ['eval_id']: ('bd213957-3ec3-5d4f-9b85-0278ceebccaf',). Skipping insertion.
Rows inserted into table 'informe_v2_pagina2' successfully.
Duplicate row found for unique columns ['eval_id']: ('bd213957-3ec3-5d4f-9b85-0278ceebccaf',). Skipping insertion.
Rows inserted into table 'informe_v2_pagina3_consumos' successfully.
Duplicate row found for unique columns ['eval_id']: ('bd213957-3ec3-5d4f-9b85-0278ceebccaf',). Skipping insertion.
Duplicate row found for unique columns ['eval_id']: ('bd213957-3ec3-5d4f-9b85-0278ceebccaf',). Skipping insertion.
Duplicate row found for unique columns ['eval_id']: ('bd213957-3ec3-5d4

/tmp/ipykernel_3362/939193779.py:34: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  informe_v2_pagina3_consumos_df = pd.concat([informe_v2_pagina3_consumos_df, informe_v2_pagina3_consumos_df_i], axis=0)


Duplicate row found for unique columns ['eval_id']: ('00200b8e-3f66-54de-8aa1-09c8ddd5d685',). Skipping insertion.
Rows inserted into table 'informe_v2_pagina1' successfully.
Duplicate row found for unique columns ['eval_id']: ('00200b8e-3f66-54de-8aa1-09c8ddd5d685',). Skipping insertion.
Rows inserted into table 'informe_v2_pagina2' successfully.
Duplicate row found for unique columns ['eval_id']: ('00200b8e-3f66-54de-8aa1-09c8ddd5d685',). Skipping insertion.
Rows inserted into table 'informe_v2_pagina3_consumos' successfully.
Duplicate row found for unique columns ['eval_id']: ('00200b8e-3f66-54de-8aa1-09c8ddd5d685',). Skipping insertion.
Duplicate row found for unique columns ['eval_id']: ('00200b8e-3f66-54de-8aa1-09c8ddd5d685',). Skipping insertion.
Duplicate row found for unique columns ['eval_id']: ('00200b8e-3f66-54de-8aa1-09c8ddd5d685',). Skipping insertion.
Duplicate row found for unique columns ['eval_id']: ('00200b8e-3f66-54de-8aa1-09c8ddd5d685',). Skipping insertion.
Duplic

/tmp/ipykernel_3362/939193779.py:30: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  informe_v2_pagina2_df = pd.concat([informe_v2_pagina2_df, informe_v2_pagina2_df_i], axis=0)


Rows inserted into table 'informe_v2_pagina1' successfully.
Rows inserted into table 'informe_v2_pagina2' successfully.
Rows inserted into table 'informe_v2_pagina3_consumos' successfully.
Duplicate row found for unique columns ['eval_id']: ('0de48209-0a22-5891-8858-3dc66cf5d079',). Skipping insertion.
Duplicate row found for unique columns ['eval_id']: ('0de48209-0a22-5891-8858-3dc66cf5d079',). Skipping insertion.
Duplicate row found for unique columns ['eval_id']: ('0de48209-0a22-5891-8858-3dc66cf5d079',). Skipping insertion.
Duplicate row found for unique columns ['eval_id']: ('0de48209-0a22-5891-8858-3dc66cf5d079',). Skipping insertion.
Duplicate row found for unique columns ['eval_id']: ('0de48209-0a22-5891-8858-3dc66cf5d079',). Skipping insertion.
Duplicate row found for unique columns ['eval_id']: ('0de48209-0a22-5891-8858-3dc66cf5d079',). Skipping insertion.
Duplicate row found for unique columns ['eval_id']: ('0de48209-0a22-5891-8858-3dc66cf5d079',). Skipping insertion.
Duplic

In [ ]:
informe_v2_pagina1_df

,eval_id,tipo_evaluacion,codigo_evaluacion,region,comuna,direccion,rol_vivienda_proyecto,tipo_vivienda,superficie_interior_util_m2,porcentaje_ahorro,letra_eficiencia_energetica_dem,demanda_calefaccion_kwh_m2_ano,demanda_enfriamiento_kwh_m2_ano,demanda_total_kwh_m2_ano,emitida_el
0,bd213957-3ec3-5d4f-9b85-0278ceebccaf,Calificación Energética,a5e79d2019,X Región de Los Lagos,Llanquihue,"Fundo Linea Nueva, Parcela 20",1014-20,"Casa Aislada, Viv. Unifam 297,63m2, Casa Llanq...",200.4,43,C,34.7,30.4,65.1,22-02-2019
1,00200b8e-3f66-54de-8aa1-09c8ddd5d685,Precalificación Energética,b15e7e2021,Región de Los Lagos,Osorno,Avenida José Ignacio Zenteno 2707,2274-513,"Departamento, 43, C41",52.6,72,A,21.7,2.3,24.0,11-06-2021
2,01183d3e-b3fe-52e0-842c-4969a788c12d,Precalificación Energética,caad5c2020,Región de Los Lagos,Osorno,"Valparaíso Nº435, Osorno",815-9,"Departamento, Piso 7, 711",44.4,58,B,19.8,4.0,23.8,29-08-2020
3,0365596d-3849-5b93-813c-d16dc71277eb,Precalificación Energética,c601c82018,X Región de Los Lagos,Osorno,"Recoleta 2253, Rahue Alto",12205-09,"Casa Aislada, SO52, 24",44.0,35,D,93.2,2.4,95.6,11-12-2018
4,0794c34f-6059-51c5-a80e-b32f938e7358,Precalificación Energética,b0596e2020,Región de Los Lagos,Osorno,"Valparaíso Nº479, Osorno",815-12,"Departamento, Piso 6, 609",38.2,56,B,20.6,14.7,35.4,03-09-2020
5,01e92e4e-f6b7-514e-a256-06264f380675,Precalificación Energética,876c2f2018,I Región de Tarapacá,Iquique,Avenida Progreso 1947,815-1,"Departamento, departamento tipo, Block A, piso...",52.2,-77,G,0.0,20.7,20.7,11-10-2018
6,2eb2a9b8-18d6-587f-bfbf-d13162b0a6d8,Precalificación Energética,83c33a2018,I Región de Tarapacá,Iquique,Avenida Progreso 1947,815-1,"Departamento, departamento tipo, Block E, piso...",52.2,27,D,0.4,10.7,11.1,13-10-2018


In [ ]:
informe_v2_pagina2_df 

,eval_id,region,comuna,direccion,rol_vivienda,tipo_vivienda,zona_termica,superficie_interior_util_m2,solicitado_por,evaluado_por,...,techo_secundario_descripcion,techo_secundario_exigencia_W_m2_K,superficie_vidriada_principal_descripcion,superficie_vidriada_principal_exigencia,superficie_vidriada_secundaria_descripcion,superficie_vidriada_secundaria_exigencia,ventilacion_rah_descripcion,ventilacion_rah_exigencia,infiltraciones_rah_descripcion,infiltraciones_rah_exigencia
0,bd213957-3ec3-5d4f-9b85-0278ceebccaf,X Región de Los Lagos,Llanquihue,"Fundo Linea Nueva, Parcela 20",1014-20,"Casa Aislada, Viv. Unifam 297,63m2, C",G,200.4,Pablo Araya Cabello,RODOLFO WIEDMAIER DELORENZO,...,Techo Techo 2 con aislación por Inter Elemento...,0.28,"Vidrio DVH con espaciador de 12mm con U= 2,85 ...",None,None,None,sin exigencia,sin exigencia,sin exigencia,sin exigencia
1,00200b8e-3f66-54de-8aa1-09c8ddd5d685,Región de Los Lagos,Osorno,Avenida José Ignacio Zenteno 2707,2274-513,"Departamento, 43, C41",G,52.6,Inmobiliaria Monte Annapurna Spa,ISABEL CLARA CORUJO GROS,...,None,0.33,Vidrio DVH con espaciador de 12mm con U= 2.85 ...,None,Vidrio DVH SGG Climaplus 6 / Argon 12 / Planit...,None,sin exigencia,sin exigencia,sin exigencia,sin exigencia
2,01183d3e-b3fe-52e0-842c-4969a788c12d,Región de Los Lagos,Osorno,"Valparaíso Nº435, Osorno",815-9,"Departamento, Piso 7, 711",G,44.4,Inmobiliaria Sur Cincuenta SpA,JOSÉ ANTONIO KOVACEVIC WALKER,...,None,0.33,"Vidrio DVH con espaciador de 6mm con U= 3,28 [...",None,None,None,sin exigencia,sin exigencia,sin exigencia,sin exigencia
3,0365596d-3849-5b93-813c-d16dc71277eb,X Región de Los Lagos,Osorno,"Recoleta 2253, Rahue Alto",12205-09,"Casa Aislada, SO52, 24",G,44.0,0,FRANCISCO GUILLERMO HARTUNG ENRÍQ,...,Techo Techo Secundario con estructura de Metal...,0.33,Vidrio Vidrio Monolítico (VM). Sin espaciador ...,None,"Vidrio DVH con espaciador de 6mm con U= 3,28 [...",None,sin exigencia,sin exigencia,sin exigencia,sin exigencia
4,0794c34f-6059-51c5-a80e-b32f938e7358,Región de Los Lagos,Osorno,"Valparaíso Nº479, Osorno",815-12,"Departamento, Piso 6, 609",G,38.2,Inmobiliaria Sur Cincuenta SpA,JOSÉ ANTONIO KOVACEVIC WALKER,...,None,0.33,"Vidrio DVH con espaciador de 6mm con U= 3,28 [...",None,None,None,sin exigencia,sin exigencia,sin exigencia,sin exigencia
5,01e92e4e-f6b7-514e-a256-06264f380675,I Región de Tarapacá,Iquique,Avenida Progreso 1947,815-1,"Departamento, departamento tipo, Block",A,52.2,Servicio de Vivienda y Urbanismo,MAURICIO ANDRÉS ELGUEDA LEZANA,...,None,0.84,Vidrio Vidrio Monolítico (VM). Sin espaciador ...,None,None,None,sin exigencia,sin exigencia,sin exigencia,sin exigencia
6,2eb2a9b8-18d6-587f-bfbf-d13162b0a6d8,I Región de Tarapacá,Iquique,Avenida Progreso 1947,815-1,"Departamento, departamento tipo, Block",A,52.2,Servicio de Vivienda y Urbanismo,MAURICIO ANDRÉS ELGUEDA LEZANA,...,None,0.84,Vidrio Vidrio Monolítico (VM). Sin espaciador ...,None,None,None,sin exigencia,sin exigencia,sin exigencia,sin exigencia


In [ ]:
informe_v2_pagina3_consumos_df

,eval_id,codigo_evaluacion,agua_caliente_sanitaria_kwh_m2,agua_caliente_sanitaria_perc,iluminacion_kwh_m2,iluminacion_per,calefaccion_kwh_m2,calefaccion_kwh_per,energia_renovable_no_convencional_kwh_m2,energia_renovable_no_convencional_per,...,aporte_fotovoltaicos_consumos_basicos_kwh,diferencia_fotovoltaica_para_consumo_kwh,aporte_solar_termica_consumos_basicos_kwh,aporte_solar_termica_agua_caliente_sanitaria_kwh,total_consumo_ep_antes_fotovoltaica_kwh,aporte_fotovoltaicos_consumos_basicos_kwh_bis,consumos_basicos_a_suplir_kwh,consumo_total_ep_obj_kwh,consumo_total_ep_ref_kwh,coeficiente_energetico_c
0,bd213957-3ec3-5d4f-9b85-0278ceebccaf,a5e79d2019,34.1,0.2,3.2,0.0,53.0,0.3,88.9,0.5,...,644.4,17172.4,0.0,0.0,18103.1,644.4,17458.7,17458.7,41689.7,0.4
1,00200b8e-3f66-54de-8aa1-09c8ddd5d685,b15e7e2021,71.7,0.6,4.0,0.0,41.3,0.4,0.0,0.0,...,0.0,0.0,0.0,0.0,6154.8,0.0,6154.8,6154.8,8565.1,0.7
2,01183d3e-b3fe-52e0-842c-4969a788c12d,caad5c2020,61.6,0.3,4.0,0.0,104.6,0.6,0.0,0.0,...,0.0,0.0,0.0,0.0,8222.3,0.0,8222.3,8222.3,7966.2,1.0
3,0365596d-3849-5b93-813c-d16dc71277eb,c601c82018,61.7,0.2,5.9,0.0,227.8,0.8,0.0,0.0,...,0.0,0.0,0.0,0.0,13005.7,0.0,13005.7,13005.7,18489.5,0.7
4,0794c34f-6059-51c5-a80e-b32f938e7358,b0596e2020,63.3,0.3,4.4,0.0,108.9,0.6,0.0,0.0,...,0.0,0.0,0.0,0.0,7315.7,0.0,7315.7,7315.7,8323.7,0.9
5,01e92e4e-f6b7-514e-a256-06264f380675,876c2f2018,25.2,0.6,3.5,0.1,0.0,0.0,15.9,0.4,...,0.0,0.0,0.0,829.0,1497.7,0.0,1497.7,1497.7,3367.3,0.4
6,2eb2a9b8-18d6-587f-bfbf-d13162b0a6d8,83c33a2018,25.2,0.6,3.5,0.1,1.0,0.0,15.9,0.3,...,0.0,0.0,0.0,829.0,1551.3,0.0,1551.3,1551.3,3393.4,0.5


In [ ]:
informe_v2_pagina4_df

,eval_id,codigo_evaluacion,mes_id,demanda_calef_viv_eval_kwh,demanda_calef_viv_eval_comment,demanda_calef_viv_ref_kwh,demanda_calef_viv_ref_comment,demanda_enfri_viv_eval_kwh,demanda_enfri_viv_eval_comment,demanda_enfri_viv_ref_kwh,demanda_enfri_viv_ref_comment,sobrecalentamiento_viv_eval_hr,sobrecalentamiento_viv_eval_comment,sobrecalentamiento_viv_ref_hr,sobrecalentamiento_viv_ref_comment,sobreenfriamiento_viv_eval_hr,sobreenfriamiento_viv_eval_comment,sobreenfriamiento_viv_ref_hr,sobreenfriamiento_viv_ref_comment
0,bd213957-3ec3-5d4f-9b85-0278ceebccaf,a5e79d2019,1,25.3,OK,88.7,OK,-1058.4,OK,-1629.9,OK,0.0,OK,0.0,OK,15.0,OK,15.0,OK
1,bd213957-3ec3-5d4f-9b85-0278ceebccaf,a5e79d2019,2,79.2,OK,193.1,OK,-890.7,OK,-1280.4,OK,0.0,OK,1.2,OK,14.0,OK,13.5,OK
2,bd213957-3ec3-5d4f-9b85-0278ceebccaf,a5e79d2019,3,136.1,OK,279.0,OK,-900.7,OK,-1006.4,OK,0.0,OK,2.2,OK,15.0,OK,12.2,OK
3,bd213957-3ec3-5d4f-9b85-0278ceebccaf,a5e79d2019,4,629.3,OK,1310.7,OK,-388.9,OK,-389.8,OK,9.0,OK,13.8,OK,4.0,OK,2.0,OK
4,bd213957-3ec3-5d4f-9b85-0278ceebccaf,a5e79d2019,5,868.9,OK,1840.5,OK,0.0,OK,0.0,OK,21.0,OK,22.5,OK,0.0,OK,0.0,OK
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
79,2eb2a9b8-18d6-587f-bfbf-d13162b0a6d8,83c33a2018,8,12.4,OK,115.0,OK,0.0,Check!,0.0,Check!,3.0,OK,6.5,OK,0.0,OK,0.0,OK
80,2eb2a9b8-18d6-587f-bfbf-d13162b0a6d8,83c33a2018,9,0.0,OK,40.6,OK,0.0,Check!,-8.3,Check!,0.0,OK,4.2,OK,0.0,OK,0.0,OK
81,2eb2a9b8-18d6-587f-bfbf-d13162b0a6d8,83c33a2018,10,0.0,OK,0.4,OK,0.0,Check!,-82.9,Check!,0.0,OK,0.0,OK,0.0,OK,0.0,OK
82,2eb2a9b8-18d6-587f-bfbf-d13162b0a6d8,83c33a2018,11,0.0,OK,0.0,OK,0.0,Check!,0.0,Check!,0.0,OK,0.0,OK,1.0,OK,0.8,OK


In [ ]:
informe_v2_pagina5_df

,eval_id,codigo_evaluacion,content
0,bd213957-3ec3-5d4f-9b85-0278ceebccaf,a5e79d2019,None
1,00200b8e-3f66-54de-8aa1-09c8ddd5d685,b15e7e2021,None
2,01183d3e-b3fe-52e0-842c-4969a788c12d,caad5c2020,None
3,0365596d-3849-5b93-813c-d16dc71277eb,c601c82018,None
4,0794c34f-6059-51c5-a80e-b32f938e7358,b0596e2020,None
5,01e92e4e-f6b7-514e-a256-06264f380675,876c2f2018,None
6,2eb2a9b8-18d6-587f-bfbf-d13162b0a6d8,83c33a2018,None


In [ ]:
informe_v2_pagina6_df

,eval_id,codigo_evaluacion,content
0,bd213957-3ec3-5d4f-9b85-0278ceebccaf,a5e79d2019,None
1,00200b8e-3f66-54de-8aa1-09c8ddd5d685,b15e7e2021,None
2,01183d3e-b3fe-52e0-842c-4969a788c12d,caad5c2020,None
3,0365596d-3849-5b93-813c-d16dc71277eb,c601c82018,None
4,0794c34f-6059-51c5-a80e-b32f938e7358,b0596e2020,None
5,01e92e4e-f6b7-514e-a256-06264f380675,876c2f2018,None
6,2eb2a9b8-18d6-587f-bfbf-d13162b0a6d8,83c33a2018,None


In [ ]:
informe_v2_pagina7_df

,eval_id,codigo_evaluacion,mandante_nombre,mandante_rut,evaluador_nombre,evaluador_rut,evaluador_rol_minvu
0,bd213957-3ec3-5d4f-9b85-0278ceebccaf,a5e79d2019,PABLO ANTONIO ARAYA CABELLO,15.934.433-9,RODOLFO WIEDMAIER DELORENZO,13.830.278-4,2254
1,00200b8e-3f66-54de-8aa1-09c8ddd5d685,b15e7e2021,INMOBILIARIA MONTE FITZ ROY SPA,76.989.209-5,ISABEL CLARA CORUJO GROS,26.952.826-5,501
2,01183d3e-b3fe-52e0-842c-4969a788c12d,caad5c2020,INMOBILIARIA SUR CINCUENTA SPA,77.015.038-8,JOSÉ ANTONIO KOVACEVIC WALKER,14.119.124-1,14070
3,0365596d-3849-5b93-813c-d16dc71277eb,c601c82018,INMOBILIARIA SOCOVESA SUR S A,96.791.150-K,FRANCISCO GUILLERMO HARTUNG ENRÍQUEZ,15.579.852-1,7016
4,0794c34f-6059-51c5-a80e-b32f938e7358,b0596e2020,INMOBILIARIA SUR CINCUENTA SPA,77.015.038-8,JOSÉ ANTONIO KOVACEVIC WALKER,14.119.124-1,14070
5,01e92e4e-f6b7-514e-a256-06264f380675,876c2f2018,SERVICIO DE VIVIENDA Y URBANIZACION I REGION,61.838.000-9,MAURICIO ANDRÉS ELGUEDA LEZANA,15.135.980-9,22540
6,2eb2a9b8-18d6-587f-bfbf-d13162b0a6d8,83c33a2018,SERVICIO DE VIVIENDA Y URBANIZACION I REGION,61.838.000-9,MAURICIO ANDRÉS ELGUEDA LEZANA,15.135.980-9,22540


In [ ]:
informe_v2_pagina1_df

,eval_id,tipo_evaluacion,codigo_evaluacion,region,comuna,direccion,rol_vivienda_proyecto,tipo_vivienda,superficie_interior_util_m2,porcentaje_ahorro,letra_eficiencia_energetica_dem,demanda_calefaccion_kwh_m2_ano,demanda_enfriamiento_kwh_m2_ano,demanda_total_kwh_m2_ano,emitida_el
0,bd213957-3ec3-5d4f-9b85-0278ceebccaf,Calificación Energética,a5e79d2019,X Región de Los Lagos,Llanquihue,"Fundo Linea Nueva, Parcela 20",1014-20,"Casa Aislada, Viv. Unifam 297,63m2, Casa Llanq...",200.4,43,C,34.7,30.4,65.1,22-02-2019
1,00200b8e-3f66-54de-8aa1-09c8ddd5d685,Precalificación Energética,b15e7e2021,Región de Los Lagos,Osorno,Avenida José Ignacio Zenteno 2707,2274-513,"Departamento, 43, C41",52.6,72,A,21.7,2.3,24.0,11-06-2021
2,01183d3e-b3fe-52e0-842c-4969a788c12d,Precalificación Energética,caad5c2020,Región de Los Lagos,Osorno,"Valparaíso Nº435, Osorno",815-9,"Departamento, Piso 7, 711",44.4,58,B,19.8,4.0,23.8,29-08-2020
3,0365596d-3849-5b93-813c-d16dc71277eb,Precalificación Energética,c601c82018,X Región de Los Lagos,Osorno,"Recoleta 2253, Rahue Alto",12205-09,"Casa Aislada, SO52, 24",44.0,35,D,93.2,2.4,95.6,11-12-2018
4,0794c34f-6059-51c5-a80e-b32f938e7358,Precalificación Energética,b0596e2020,Región de Los Lagos,Osorno,"Valparaíso Nº479, Osorno",815-12,"Departamento, Piso 6, 609",38.2,56,B,20.6,14.7,35.4,03-09-2020
5,01e92e4e-f6b7-514e-a256-06264f380675,Precalificación Energética,876c2f2018,I Región de Tarapacá,Iquique,Avenida Progreso 1947,815-1,"Departamento, departamento tipo, Block A, piso...",52.2,-77,G,0.0,20.7,20.7,11-10-2018
6,2eb2a9b8-18d6-587f-bfbf-d13162b0a6d8,Precalificación Energética,83c33a2018,I Región de Tarapacá,Iquique,Avenida Progreso 1947,815-1,"Departamento, departamento tipo, Block E, piso...",52.2,27,D,0.4,10.7,11.1,13-10-2018


insert_unique_rows_from_dataframe(db_file_path, 'informe_v2_pagina1', informe_v2_pagina1_df, unique_columns=['eval_id'])
insert_unique_rows_from_dataframe(db_file_path, 'informe_v2_pagina2', informe_v2_pagina2_df, unique_columns=['eval_id'])
insert_unique_rows_from_dataframe(db_file_path, 'informe_v2_pagina3_consumos', informe_v2_pagina3_consumos_df, unique_columns=['eval_id'])
insert_unique_rows_from_dataframe(db_file_path, 'informe_v2_pagina3_envolvente', informe_v2_pagina3_envolvente_df, unique_columns=['eval_id'])
insert_unique_rows_from_dataframe(db_file_path, 'informe_v2_pagina4', informe_v2_pagina4_df, unique_columns=['eval_id'])
insert_unique_rows_from_dataframe(db_file_path, 'informe_v2_pagina5', informe_v2_pagina5_df, unique_columns=['eval_id'])
insert_unique_rows_from_dataframe(db_file_path, 'informe_v2_pagina6', informe_v2_pagina6_df, unique_columns=['eval_id'])
insert_unique_rows_from_dataframe(db_file_path, 'informe_v2_pagina7', informe_v2_pagina7_df, unique_columns=['eval_id'])

In [ ]:
## Save Excel

In [ ]:
excel_file_path = './reports_summary.xlsx'

In [ ]:
replace_sheet_content(excel_file_path, 'summary', summary_df)
replace_sheet_content(excel_file_path, 'informe_v2_pagina1', informe_v2_pagina1_df)
replace_sheet_content(excel_file_path, 'informe_v2_pagina2', informe_v2_pagina2_df)
replace_sheet_content(excel_file_path, 'informe_v2_pagina3_consumos', informe_v2_pagina3_consumos_df)
replace_sheet_content(excel_file_path, 'informe_v2_pagina3_envolvente', informe_v2_pagina3_envolvente_df)
replace_sheet_content(excel_file_path, 'informe_v2_pagina4', informe_v2_pagina4_df)
replace_sheet_content(excel_file_path, 'informe_v2_pagina5', informe_v2_pagina5_df)
replace_sheet_content(excel_file_path, 'informe_v2_pagina6', informe_v2_pagina6_df)
replace_sheet_content(excel_file_path, 'informe_v2_pagina7', informe_v2_pagina7_df)

Data replaced in sheet 'summary' in './reports_summary.xlsx'
Data replaced in sheet 'informe_v2_pagina1' in './reports_summary.xlsx'
Data replaced in sheet 'informe_v2_pagina2' in './reports_summary.xlsx'
Data replaced in sheet 'informe_v2_pagina3_consumos' in './reports_summary.xlsx'
Data replaced in sheet 'informe_v2_pagina3_envolvente' in './reports_summary.xlsx'
Data replaced in sheet 'informe_v2_pagina4' in './reports_summary.xlsx'
Data replaced in sheet 'informe_v2_pagina5' in './reports_summary.xlsx'
Data replaced in sheet 'informe_v2_pagina6' in './reports_summary.xlsx'
Data replaced in sheet 'informe_v2_pagina7' in './reports_summary.xlsx'
